In [21]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from keras import layers
import utils as ut
import numpy as np
from typing import Dict, Optional, Tuple

In [22]:
def get_model():
    model = tf.keras.models.Sequential(
        [
            layers.Input(shape=(8,)),  
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),  
            layers.Dense(1)  
        ]
    )
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
    return model

In [23]:
class CifarClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test):
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""

        # Update local model parameters
        self.model.set_weights(parameters)

        # Get hyperparameters for this round
        batch_size: int = config["batch_size"]
        epochs: int = config["local_epochs"]

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
        )

        # Return updated model parameters, number of examples trained, and results
        parameters_prime = self.model.get_weights()
        num_examples_train = len(self.x_train)
        results = {
            "mse": history.history["mse"][-1],  # Use the last epoch's MSE
            "mae": history.history["mae"][-1],  # Use the last epoch's MAE
            "val_mse": history.history["val_mse"][-1],  # Use the last epoch's validation MSE
            "val_mae": history.history["val_mae"][-1],  # Use the last epoch's validation MAE
        }

        return parameters_prime, num_examples_train, results

    def get_parameters(self):
        """Get the current parameters of the local model."""
        return self.model.get_weights()

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""
        self.model.set_weights(parameters)
        loss = tf.keras.losses.mean_squared_error(self.y_test, self.model.predict(self.x_test)).numpy().mean().item()
        mae = tf.keras.metrics.mean_absolute_error(self.y_test, self.model.predict(self.x_test)).numpy().mean().item()
        print("*************LOSS******************",loss)
        print("************MAE********************",mae)
        return loss, len(self.x_train), {"mae": mae}

    def get_weights(self):
        """Get the current weights of the local model."""
        return self.model.get_weights()

    def set_weights(self, weights):
        """Set the weights of the local model."""
        self.model.set_weights(weights)

In [24]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(2,4,0)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8080",
client=client,
)

INFO flwr 2023-11-23 01:51:07,977 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:51:07,996 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:51:07,998 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/50
26/26 [==============================] - 1s 11ms/step - loss: 4.1562 - mae: 1.6719 - mse: 4.1562 - val_loss: 2.4914 - val_mae: 1.1900 - val_mse: 2.4914
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 1.5503 - mae: 0.9349 - mse: 1.5503 - val_loss: 1.1124 - val_mae: 0.8700 - val_mse: 1.1124
Epoch 3/50
26/26 [==============================] - 0s 3ms/step - loss: 1.1249 - mae: 0.8648 - mse: 1.1249 - val_loss: 1.0047 - val_mae: 0.8014 - val_mse: 1.0047
Epoch 4/50
26/26 [==============================] - 0s 3ms/step - loss: 1.0038 - mae: 0.8009 - mse: 1.0038 - val_loss: 0.8937 - val_mae: 0.7643 - val_mse: 0.8937
Epoch 5/50
26/26 [==============================] - 0s 3ms/step - loss: 0.8918 - mae: 0.7506 - mse: 0.8918 - val_loss: 0.7939 - val_mae: 0.7145 - val_mse: 0.7939
Epoch 6/50
26/26 [==============================] - 0s 3ms/step - loss: 0.7889 - mae: 0.7002 - mse: 0.7889 - val_loss: 0.7040 - val_mae: 0.6600 - val_mse: 0.7040
Epoch 7/50
26/26 [=========

DEBUG flwr 2023-11-23 01:53:46,384 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 01:53:46,386 | app.py:215 | Disconnect and shut down


In [25]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(0,4,0)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8081",
client=client,
)

INFO flwr 2023-11-23 01:54:53,705 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:54:53,724 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:54:53,727 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/50
26/26 [==============================] - 1s 13ms/step - loss: 0.4537 - mae: 0.4826 - mse: 0.4537 - val_loss: 0.3665 - val_mae: 0.4537 - val_mse: 0.3665
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 0.3367 - mae: 0.4105 - mse: 0.3367 - val_loss: 0.3435 - val_mae: 0.4311 - val_mse: 0.3435
Epoch 3/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3146 - mae: 0.3933 - mse: 0.3146 - val_loss: 0.3332 - val_mae: 0.4147 - val_mse: 0.3332
Epoch 4/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3073 - mae: 0.3841 - mse: 0.3073 - val_loss: 0.3307 - val_mae: 0.4100 - val_mse: 0.3307
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3070 - mae: 0.3844 - mse: 0.3070 - val_loss: 0.3351 - val_mae: 0.4170 - val_mse: 0.3351
Epoch 6/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3034 - mae: 0.3806 - mse: 0.3034 - val_loss: 0.3371 - val_mae: 0.4185 - val_mse: 0.3371
Epoch 7/50
26/26 [=========

DEBUG flwr 2023-11-23 01:57:16,659 | connection.py:139 | gRPC channel closed
INFO flwr 2023-11-23 01:57:16,661 | app.py:215 | Disconnect and shut down


In [26]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(-2,4,0)

client = CifarClient(get_model(), x_train, y_train, x_test, y_test)

history = fl.client.start_numpy_client(
server_address="127.0.0.1:8082",
client=client,
)

INFO flwr 2023-11-23 01:59:51,626 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-23 01:59:51,646 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-23 01:59:51,648 | connection.py:42 | ChannelConnectivity.READY


Epoch 1/50
26/26 [==============================] - 1s 12ms/step - loss: 1.4968 - mae: 0.7870 - mse: 1.4968 - val_loss: 0.6012 - val_mae: 0.5740 - val_mse: 0.6012
Epoch 2/50
26/26 [==============================] - 0s 4ms/step - loss: 0.4842 - mae: 0.5046 - mse: 0.4842 - val_loss: 0.4970 - val_mae: 0.5122 - val_mse: 0.4970
Epoch 3/50
26/26 [==============================] - 0s 4ms/step - loss: 0.4146 - mae: 0.4628 - mse: 0.4146 - val_loss: 0.4510 - val_mae: 0.4866 - val_mse: 0.4510
Epoch 4/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3843 - mae: 0.4446 - mse: 0.3843 - val_loss: 0.4265 - val_mae: 0.4667 - val_mse: 0.4265
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3610 - mae: 0.4284 - mse: 0.3610 - val_loss: 0.4072 - val_mae: 0.4588 - val_mse: 0.4072
Epoch 6/50
26/26 [==============================] - 0s 4ms/step - loss: 0.3437 - mae: 0.4183 - mse: 0.3437 - val_loss: 0.3883 - val_mae: 0.4424 - val_mse: 0.3883
Epoch 7/50
26/26 [=========

DEBUG flwr 2023-11-23 02:02:12,512 | connection.py:139 | gRPC channel closed


 2.4310128688812256
************MAE******************** 1.2073485851287842


INFO flwr 2023-11-23 02:02:12,513 | app.py:215 | Disconnect and shut down
